In [1]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))
from os import remove


from datetime import date, timedelta
import codecs
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

from os.path import join, basename

from bulletin import default_input, default_output, root
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime


today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)

notifica = Notifica()
notifica.load()
notifica.df


---------------------------------------------------------------

root: C:\sesa\dev\bulletin
parent: C:\sesa\dev
default_input: C:\sesa\dev\input
default_output: C:\sesa\dev\output

---------------------------------------------------------------



,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_less,hash_more,hash_mae,hash_nasc
0,66722,2021-06-14,2021-06-12,1.0,1.0,2021-06-14,None,2.0,1.0,3.0,2021-06-15,3.0,700006994025602,05823228988,NaN,0.0,LUCIMARA GONCALVES DA SILVA OLIVEIRA,2.0,1978-10-06,INES BARBOSA DA SILVA,99.0,5.0,,NaN,NaN,1.0,,41.0,412810.0,8.75095e+07,Rua das Petúnias,Parque das Jaboticabeiras,41.0,412810.0,2021-06-11,2021-06-11,2021-06-11,2021-06-11,-1.000000e+00,12.0,167.0,2.0,,2021-06-13,2021-06-14,42.0,LUCIMARAGONCALVESDASILVAOLIVEIRA42412810,LUCIMARAGONCALVESDASILVAOLIVEIRA42412810,NILDACASTORINACORREIACRUZ45412710,NILDACASTORINACORREIACRUZ47412710,LUCIMARAGONCALVESDASILVAOLIVEIRAINESBARBOSADAS...,LUCIMARAGONCALVESDASILVAOLIVEIRA06101978
1,66776,2021-06-14,2021-06-14,1.0,1.0,2021-06-14,None,2.0,1.0,3.0,2021-06-03,3.0,705007851382454,50615556949,NaN,0.0,JOAO BARSOTTI,1.0,1963-12-21,MARIA ABE BARSOTTI,99.0,5.0,,NaN,NaN,1.0,,41.0,410150.0,8.615e+07,Rua Getúlio Vargas,Fátima,41.0,410150.0,2021-06-12,2021-06-14,2020-04-17,2020-04-19,-1.000000e+00,12.0,167.0,2.0,,2021-06-08,2021-06-14,57.0,JOAOBARSOTTI57410150,JOAOBARSOTTI57410150,ELIANADASILVA42411810,ELIANADASILVA44411810,JOAOBARSOTTIMARIAABEBARSOTTI,JOAOBARSOTTI21121963
2,66777,2021-06-14,2021-03-23,1.0,1.0,2021-06-14,None,2.0,1.0,3.0,2020-12-16,3.0,702706615492460,05141717909,NaN,NaN,LUCIANA SOARES SANTOS,2.0,1982-12-10,MARIA HELENA SOARES SANTOS,99.0,5.0,,7632.0,NaN,1.0,,41.0,411370.0,8.37058e+07,JOSE BERNARDO,Capela Velha,41.0,411370.0,2021-06-12,2021-06-12,NaT,2021-06-13,-1.000000e+00,12.0,167.0,2.0,,2021-06-06,2021-06-12,38.0,LUCIANASOARESSANTOS38411370,LUCIANASOARESSANTOS38411370,SALETELOURDESREBESCHINIDASILVA66412860,SALETELOURDESREBESCHINIDASILVA68412860,LUCIANASOARESSANTOSMARIAHELENASOARESSANTOS,LUCIANASOARESSANTOS10121982
3,66824,2021-06-14,2021-06-14,2.0,1.0,2021-06-14,None,2.0,1.0,1.0,2021-06-14,3.0,701209096813716,03545286967,NaN,0.0,ANDRE VIECELLI,1.0,1983-01-07,MARIA ELENA KIELTIKA VIECELLI,99.0,5.0,BANCARIO,NaN,NaN,1.0,,41.0,410480.0,8.6975e+07,RUA EUCLIDES DA CUNHA,PQ. SAO PAULO,41.0,410480.0,2021-06-07,2021-06-05,2021-06-07,2021-06-07,2.110020e+11,9.0,27.0,1.0,DASA CASCAVEL,2021-06-05,2021-06-05,38.0,ANDREVIECELLI38410480,ANDREVIECELLI38410480,MARCIANOPEREIRALUCAS33410510,MARCIANOPEREIRALUCAS35410510,ANDREVIECELLIMARIAELENAKIELTIKAVIECELLI,ANDREVIECELLI07011983
4,66864,2021-06-14,2021-06-14,1.0,1.0,2021-06-14,None,2.0,1.0,3.0,2021-06-14,3.0,702301145150517,78173850925,NaN,0.0,JUNIOR CESER FILIPPI,1.0,1971-07-15,ALDERINA BORTOLANZA FILIPPI,1.0,0.0,,0.0,NaN,1.0,,41.0,412215.0,8.534e+07,sete quedas,campo do bugre,41.0,412215.0,2021-06-08,2021-06-14,2021-06-10,2021-06-11,-1.000000e+00,12.0,184.0,6.0,,2021-06-10,2021-06-14,49.0,JUNIORCESERFILIPPI49412215,JUNIORCESERFILIPPI49412215,VALDOMIRADASILVACORDEIRO83412625,VALDOMIRADASILVACORDEIRO85412625,JUNIORCESERFILIPPIALDERINABORTOLANZAFILIPPI,JUNIORCESERFILIPPI15071971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191702,4302702,2020-11-04,2021-06-18,2.0,1.0,2021-06-18,None,2.0,1.0,3.0,NaT,3.0,702602288882048,09399626911,NaN,NaN,FRANCIELE CIAN,2.0,1995-03-20,MARINA APARECIDA BIAGIO CIAN,99.0,NaN,,NaN,NaN,1.0,,41.0,410150.0,NaN,quetzal,vila sampaio,41.0,410150.0,2020-11-04,2020-11-04,2020-11-06,2020-11-07,2.016010e+11,

In [6]:
exames = pd.read_csv(join(root,'resources','tables','exame.csv'))
exames.columns = ['exame','exame_name']
df = pd.merge(notifica.df,exames,how='left',on='exame')

In [10]:
df[['id','exame','exame_name']].groupby(['exame','exame_name']).count().sort_values('id',ascending=False)

,,id
exame,exame_name,
27.0,"COVID-19, Biologia Molecular",740466
167.0,Teste Rápido Antígeno (Imunocromatografia),213239
1.0,Vírus Respiratório,79563
181.0,Teste Molecular (RT-PCR),18892
179.0,Teste Rápido IgM/IgG não discriminatório (Imunocromatografia),16411
...,...,...
147.0,SARS-CoV-2 IgM/IgG AB RAPID TEST,1
124.0,COVID-19 IgG/IgM DUO (NANOENTEK),1
114.0,ATELLICA IM SARS-CoV-2 TOTAL (COV2T),1


In [16]:
df.loc[df[167]]

KeyError: 167

In [1]:
import sys
from pathlib import Path
from os import getcwd

from datetime import date, timedelta

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
from sys import exit

In [3]:
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input


---------------------------------------------------------------

root: C:\sesa\dev\bulletin
parent: C:\sesa\dev
default_input: C:\sesa\dev\input
default_output: C:\sesa\dev\output

---------------------------------------------------------------



In [4]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [8]:
# tb_pacientes = TbPacientes()
# # tb_pacientes.update()
# casost = tb_pacientes.get()
casost = pd.read_pickle('tb_pacientes.pkl')#tb_pacientes.get()
casost['nome'] = casost['nome'].apply(lambda nome: ''.join(filter(lambda x: ('A' <= x <= 'Z') or ( x == ' '), nome)))
casost = casost.sort_values(['dt_notificacao','nome','idade'])
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
# casost = casost.loc[casost['dt_notificacao'] <= '2021-05-27'].copy()
print(f"casost len: {len(casost)}\n")
casost.groupby('obito')[['sexo']].count()

casost len: 1162966

casost len: 1162966



,sexo
obito,
NAO,1134119
SIM,28847


In [9]:
casost

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito
8951,8838,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-04-03 00:00:00,Notificações,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,47,NaT,NaT,48,46,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,NaN
11016,10932,410690,410690,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,43,NaT,NaT,44,42,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU12032020,NaN
11017,10933,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,15,NaT,NaT,16,14,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR12032020,NaN
11550,11470,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,NaN,07/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,26,NaT,NaT,27,25,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE16032020,NaN
11572,11492,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,NaN,08/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,26,NaT,NaT,27,25,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN18032020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603795,1252007,412440,412440,ZONIRA FERNANDES BUENO MINETTO,F,54,SANTO ANTONIO DO SUDOESTE,SANTO ANTONIO DO SUDOESTE,167,2021-06-15,2021-06-16,2021-06-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54,NaT,NaT,55,53,ZONIRAFERNANDESBUENOMINETTO54SANTOANTONIODOSUD...,ZONIRAFERNANDESBUENOMINETTO53SANTOANTONIODOSUD...,ZONIRAFERNANDESBUENOMINETTO55SANTOANTONIODOSUD...,ZONIRAFERNANDESBUENOMINETTO54SANTOANTONIODOSUD...,ZONIRAFERNANDESBUENOMINETTO53SANTOANTONIODOSUD...,ZONIRAFERNANDESBUENOMINETTO55SANTOAN

In [10]:
casost.groupby('status')[['sexo']].count()

,sexo
status,
Recuperado,818584
recuperado,2


In [11]:
casosct = casost.copy()

casosct['evolucao'] = 3
casosct.loc[casosct['obito']=='SIM','evolucao'] = 2
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'evolucao'] = 1

In [12]:
casosct.loc[casosct['obito']=='SIM','data_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_obito'] 
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_atualizacao']

In [13]:
casosct.loc[casosct['obito']=='SIM','data_com_evolucao'] = casosct.loc[casosct['obito']=='SIM','dt_com_obito'] 
casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']=='SIM')&(casosct['data_com_evolucao'].isna()),'dt_obito']
casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'data_com_evolucao'] = casosct.loc[(casosct['obito']!='SIM')&(casosct['status'].notna()),'dt_com_recuperado'] 

In [14]:
casosct.groupby('evolucao')[['sexo']].count()

,sexo
evolucao,
1,818586
2,28847
3,315533


In [15]:
casosct['uf_resid'] = 'PR'
casosct['ibge_res_pr'] = casosct['ibge_res_pr'].apply(int)
casosct.loc[casosct['ibge_res_pr']==999999,'uf_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[-1])
casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'] = casosct.loc[casosct['ibge_res_pr']==999999,'mun_resid'].apply(lambda x: x.split('/')[0])

casos_fora = pd.merge(casosct.loc[casosct['ibge_res_pr']==999999].drop(columns=['ibge_res_pr']),municipios[['ibge','mun_resid','uf']].rename(columns={'uf':'uf_resid','ibge':'ibge_res_pr'}), on=['mun_resid','uf_resid'], how='left')
# casos_fora.loc[casos_fora['ibge_res_pr'].isna(),'excluir'] = 'SIM'
casos_fora.loc[casos_fora['ibge_res_pr'].isna(),'ibge_res_pr'] = 999999
casos_fora = casos_fora.set_index('identificacao')
casosct = casosct.set_index('identificacao')
casosct.update(casos_fora)

casosct = casosct.reset_index()

casosct['ibge_res_pr'] = casosct['ibge_res_pr'].apply(int)
casosct['idade'] = casosct['idade'].astype(int)
casosct

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais,idade_menos,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag,hash_obito,data_evolucao,data_com_evolucao,uf_resid
0,8838,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,CIANORTE,CIANORTE,LACEN,2020-03-12,2020-03-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,2020-04-03 00:00:00,Notificações,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,23.0,47.0,NaT,NaT,48.0,46.0,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA47CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA46CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA48CIANORTE,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020,NaN,2020-04-03 00:00:00,NaT,PR
1,10932,410690,410690,FLAMARION PETRIV DE ABREU,M,43,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,23.0,43.0,NaT,NaT,44.0,42.0,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU43CURITIBA,FLAMARIONPETRIVDEABREU42CURITIBA,FLAMARIONPETRIVDEABREU44CURITIBA,FLAMARIONPETRIVDEABREU12032020,NaN,NaN,NaT,PR
2,10933,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,CURITIBA,CURITIBA,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,NaN,43897,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,23.0,15.0,NaT,NaT,16.0,14.0,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR15CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR14CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR16CURITIBA,FLAMARIONPETRIVDEABREUJUNIOR12032020,NaN,NaN,NaT,PR
3,11470,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,NaN,07/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,14.0,26.0,NaT,NaT,27.0,25.0,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE26CURITIBA,GABRIELAFUGANTICASAGRANDE25CURITIBA,GABRIELAFUGANTICASAGRANDE27CURITIBA,GABRIELAFUGANTICASAGRANDE16032020,NaN,NaN,NaT,PR
4,11492,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,CURITIBA,CURITIBA,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,NaN,08/03/2020 a 10/03/2020,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Recuperado,NaN,NaN,NaN,NaN,1.0,NaN,NaN,14.0,26.0,NaT,NaT,27.0,25.0,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN26CURITIBA,GABRIELASCHNEIDERGUGELMIN25CURITIBA,GABRIELASCHNEIDERGUGELMIN27CURITIBA,GABRIELASCHNEIDERGUGELMIN18032020,NaN,NaN,NaT,PR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162961,1252007,412440,412440,ZONIRA FERNANDES BUENO MINETTO,F,54,SANTO ANTONIO DO SUDOESTE,SANTO ANTONIO DO SUDOESTE,167,2021-06-15,2021-06-16,2021-06-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAO,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,54.0,NaT,NaT,55.0,53.0,ZONIRAFERNANDESBUENOMINETTO54SANTOANTONIODOSUD...,ZONIRAFERNANDESBUENOMINETTO53SANTOANTONIODOSUD...,ZONIRAFERNANDESBUENOMINETTO55SANTOANTONIODOS

In [16]:
casosct.groupby(['evolucao'])[['sexo']].count()

,sexo
evolucao,
1.0,818586
2.0,28847
3.0,315533


In [17]:
casosct['id_notifica'] = -1
casosct['ibge_resid'] = casosct['ibge_res_pr']
casosct['ibge_atend'] = casosct['ibge_atend_pr'].apply(int)
casosct['comunicacao'] = casosct['dt_notificacao']
casosct['is'] = casosct['dt_inicio_sintomas']

casosct = casosct[['identificacao', 'id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao', 'data_com_evolucao']]
casosct

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaN,1.0,2020-04-03 00:00:00,NaT
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,1.0,NaN,NaT
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,1.0,NaN,NaT
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,1.0,NaN,NaT
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,1.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162961,1252007,-1,PR,412440,412440,ZONIRA FERNANDES BUENO MINETTO,F,54,167,2021-06-15,2021-06-16,2021-06-10 00:00:00,3.0,NaT,NaT
1162962,1249899,-1,PR,410830,410830,ZULEIKA TEREZINHA TORRES PEREIRA,F,37,181,2021-06-14,2021-06-16,2021-06-11 00:00:00,3.0,NaT,NaT
1162963,1254288,-1,PR,410430,410430,ZULMIRA BERANISE DE OLIVEIRA,F,50,1,2021-04-20,2021-06-16,2021-04-14 00:00:00,1.0,2021-05-21 00:00:00,2021-06-16
1162964,1249900,-1,PR,411722,411722,ZUNILDA SARACHO SANTIN,F,49,27,2021-06-14,2021-06-16,2021-06-04 00:00:00,3.0,NaT,NaT


In [18]:
casos_confirmados = casosct.copy()
casos_confirmados

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaN,1.0,2020-04-03 00:00:00,NaT
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,1.0,NaN,NaT
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,1.0,NaN,NaT
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,1.0,NaN,NaT
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,1.0,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162961,1252007,-1,PR,412440,412440,ZONIRA FERNANDES BUENO MINETTO,F,54,167,2021-06-15,2021-06-16,2021-06-10 00:00:00,3.0,NaT,NaT
1162962,1249899,-1,PR,410830,410830,ZULEIKA TEREZINHA TORRES PEREIRA,F,37,181,2021-06-14,2021-06-16,2021-06-11 00:00:00,3.0,NaT,NaT
1162963,1254288,-1,PR,410430,410430,ZULMIRA BERANISE DE OLIVEIRA,F,50,1,2021-04-20,2021-06-16,2021-04-14 00:00:00,1.0,2021-05-21 00:00:00,2021-06-16
1162964,1249900,-1,PR,411722,411722,ZUNILDA SARACHO SANTIN,F,49,27,2021-06-14,2021-06-16,2021-06-04 00:00:00,3.0,NaT,NaT


In [19]:
casos_confirmados['is'] = casos_confirmados['is'].apply(lambda x: str(x)[:10] if not str(x).lower() in ['none','nat','nan'] else 'none')
casos_confirmados.loc[casos_confirmados['is'].str.contains('/'),'is2'] = casos_confirmados.loc[casos_confirmados['is'].str.contains('/'),'is'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
casos_confirmados.loc[casos_confirmados['is'].str.contains('-'),'is2'] = casos_confirmados.loc[casos_confirmados['is'].str.contains('-'),'is'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

casos_confirmados['comunicacao'] = casos_confirmados['comunicacao'].apply(str)
casos_confirmados.loc[casos_confirmados['comunicacao'].str.contains('/'),'com'] = casos_confirmados.loc[casos_confirmados['comunicacao'].str.contains('/'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='raise'))
casos_confirmados.loc[casos_confirmados['comunicacao'].str.contains('-'),'com'] = casos_confirmados.loc[casos_confirmados['comunicacao'].str.contains('-'),'comunicacao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))

casos_confirmados['data_evolucao'] = casos_confirmados['data_evolucao'].apply(str)
casos_confirmados.loc[casos_confirmados['data_evolucao'].str.contains('/'),'evo'] = casos_confirmados.loc[casos_confirmados['data_evolucao'].str.contains('/'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%d/%m/%Y',errors='coerce'))
casos_confirmados.loc[casos_confirmados['data_evolucao'].str.contains('-'),'evo'] = casos_confirmados.loc[casos_confirmados['data_evolucao'].str.contains('-'),'data_evolucao'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='coerce'))

casos_confirmados['id_notifica'] = casos_confirmados['id_notifica'].apply(int)
casos_confirmados['evolucao'] = casos_confirmados['evolucao'].apply(int)
casos_confirmados['is'] = casos_confirmados['is2']
casos_confirmados['comunicacao'] = casos_confirmados['com']
casos_confirmados['data_evolucao'] = casos_confirmados['evo'].apply(lambda x: pd.to_datetime(x))

In [20]:
casos_confirmados['id_notifica'] = casos_confirmados['id_notifica'].astype(int)
casos_confirmados['ibge_resid'] = casos_confirmados['ibge_resid'].astype(int)
casos_confirmados['ibge_atend'] = casos_confirmados['ibge_atend'].astype(int)
casos_confirmados['idade'] = casos_confirmados['idade'].astype(int)
casos_confirmados['evolucao'] = casos_confirmados['evolucao'].astype(int)

casos_confirmados['hash'] = ( casos_confirmados['nome'].apply(normalize_hash) +
               casos_confirmados['idade'].astype(str) +
               casos_confirmados['ibge_resid'].apply(str) )

casos_confirmados['hash_less'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(x-1)) +
                    casos_confirmados['ibge_resid'].apply(str) )

casos_confirmados['hash_more'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(x+1)) +
                    casos_confirmados['ibge_resid'].apply(str) )

casos_confirmados['hash_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].astype(str) +
                    casos_confirmados['ibge_atend'].apply(str) )

casos_confirmados['hash_less_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(x-1)) +
                    casos_confirmados['ibge_atend'].apply(str) )

casos_confirmados['hash_more_atend'] = (casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['idade'].apply(lambda x: str(x+1)) +
                    casos_confirmados['ibge_atend'].apply(str) )

casos_confirmados['hash_diag'] = ( casos_confirmados['nome'].apply(normalize_hash) +
                    casos_confirmados['dt_diag'].apply(date_hash) )

In [21]:
casos_confirmados[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend','hash_diag']]

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaN,1,2020-04-03,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07 00:00:00,1,NaT,NaT,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08 00:00:00,1,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11 00:00:00,1,NaT,NaT,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE16032020
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15 00:00:00,1,NaT,NaT,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN18032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162961,1252007,-1,PR,412440,412440,ZONIRA FERNANDES BUENO MINETTO,F,54,167,2021-06-15,2021-06-16,2021-06-10 00:00:00,3,NaT,NaT,ZONIRAFERNANDESBUENOMINETTO54412440,ZONIRAFERNANDESBUENOMINETTO53412440,ZONIRAFERNANDESBUENOMINETTO55412440,ZONIRAFERNANDESBUENOMINETTO54412440,ZONIRAFERNANDESBUENOMINETTO53412440,ZONIRAFERNANDESBUENOMINETTO55412440,ZONIRAFERNANDESBUENOMINETTO15062021
1162962,1249899,-1,PR,410830,410830,ZULEIKA TEREZINHA TORRES PEREIRA,F,37,181,2021-06-14,2021-06-16,2021-06-11 00:00:00,3,NaT,NaT,ZULEIKATEREZINHATORRESPEREIRA37410830,ZULEIKATEREZINHATORRESPEREIRA36410830,ZULEIKATEREZINHATORRESPEREIRA38410830,ZULEIKATEREZINHATORRESPEREIRA37410830,ZULEIKATEREZINHATORRESPEREIRA36410830,ZULEIKATEREZINHATORRESPEREIRA38410830,ZULEIKATEREZINHATORRESPEREIRA14062021
1162963,1254288,-1,PR,410430,410430,ZULMIRA BERANISE DE OLIVEIRA,F,50,1,2021-04-20,2021-06-16,2021-04-14 00:00:00,1,2021-05-21,2021-06-16,ZULMIRABERANISEDEOLIVEIRA50410430,ZULMIRABERANISEDEOLIVEIRA49410430,ZULMIRABERANISEDEOLIVEIRA51410430,ZULMIRABERANISEDEOLIVEIRA50410430,ZULMIRABERANISEDEOLIVEIRA49410430,ZULMIRABERANISEDEOLIVEIRA51410430,ZULMIRABERANISEDEOLIVEIRA20042021
1162964,1249900,-1,PR,411722,411722,ZUNILDA SARACHO SANTIN,F,49,27,2021-06-14,2021-06-16,2021-06-04 00:00:00,3,NaT,NaT,ZUNILDASARACHOSANTIN49411722,ZUNILDASARACHOSANTIN48411722,ZUNILDASARACHOSANTIN50411722,ZUNILDASARACHOSANTIN49411722,ZUNILDASARACHOSANTIN48411722,ZUNILDASARACHOSANTIN50411722,ZUNILDASARACHOSANTIN14062021


In [34]:
# casos_confirmados['is'] = casos_confirmados['is'].apply(lambda x: str(x)[:10] if not str(x).lower() in ['none','nat','nan'] else 'none')
casos_confirmados.loc[casos_confirmados['is'].str.contains('-'),'is'] = casos_confirmados.loc[casos_confirmados['is'].str.contains('-'),'is'].apply(lambda x: pd.to_datetime(x,format='%Y-%m-%d',errors='raise'))
casos_confirmados.loc[casos_confirmados['is']=='none','is'] = pd.NaT

In [39]:
casos_confirmados['is'] = casos_confirmados['is'].apply(pd.to_datetime)

In [40]:
casos_confirmados.dtypes

identificacao                 int64
id_notifica                   int32
uf_resid                     object
ibge_resid                    int32
ibge_atend                    int32
nome                         object
sexo                         object
idade                         int32
laboratorio                  object
dt_diag              datetime64[ns]
comunicacao          datetime64[ns]
is                   datetime64[ns]
evolucao                      int32
data_evolucao        datetime64[ns]
data_com_evolucao    datetime64[ns]
is2                          object
com                  datetime64[ns]
evo                          object
hash                         object
hash_less                    object
hash_more                    object
hash_atend                   object
hash_less_atend              object
hash_more_atend              object
hash_diag                    object
dtype: object

In [41]:
cc = CasosConfirmados()
cc.df = casos_confirmados
cc.save('cc_16_06_2021',replace=True)

databases: ['cc_11_06_2021', 'cc_15_06_2021', 'cc_16_06_2021', 'cc_17_06_2021']
Load C:\sesa\dev\bulletin\database\casos_confirmados\cc_16_06_2021.pkl
saving Casos Confirmados to pkl time elepsed:00:00:16
